In [9]:
import sys
import pandas as pd
sys.path.append("..")

from chemspace.Dataset.DatasetBuilder import DatasetBuilder

Let's get the synonyms into one column of strings, although they must not be comma-separated, to avoid problems later in the week

Let's find the data

In [10]:
# Instantiate class with previously generated Dataframe (CSV)
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/Dataset.csv')
#DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/CIDs.csv')
# Display dataset
DB.CIDs
DB.dataset

,CID,SMILES
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
1,3,C1=CC(C(C(=C1)C(=O)O)O)O
2,4,CC(CN)O
3,5,C(C(=O)COP(=O)(O)O)N
4,6,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl
...,...,...
327752,168010365,CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...
327753,168010381,CCCC/C=C/C=C/CC1C(CC(=O)C2=C(C(=C)OC(=O)C2(O1)...
327754,168010385,CCCCC[C@@H]1CC(=NN(O1)C(=O)CCOC(=O)[C@H](C)OC(...
327755,168010395,CC[C@@H](C)[C@H](C(=O)N1[C@H]2C[C@@H](CC[C@H]2...


In [28]:
import gzip
content = {
    "CID": [],
    "Synonym": []
}
with gzip.open('../chemspace/Dataset/Data/CID-Synonym-filtered.gz', 'rt') as zipfile:
    for i, line in enumerate(zipfile):
        CID, synonym = line.split()[0], " ".join(line.split()[1:])
        content['CID'].append(int(CID))
        content['Synonym'].append(synonym)
print(len(content['CID']))

100007042


In [29]:
syn_DF = pd.DataFrame(content)
syn_DF.head()

,CID,Synonym
0,1,(+/-)-acetylcarnitine
1,1,(3-CARBOXY-2-HYDROXYPROPYL)TRIMETHYLAMMONIUM H...
2,1,07OP6H4V4A
3,1,"1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-t..."
4,1,"1-Propanaminium, 2-(acetoxy)-3-carboxy-N,N,N-t..."


In [30]:
syn_DF = syn_DF.groupby('CID')['Synonym'].apply('; '.join).reset_index()
syn_DF['Number_of_Synonyms'] = syn_DF['Synonym'].apply(lambda x: len(x.split('; ')))

In [38]:
len(syn_DF['CID'])

58001485

In [35]:
syn_DF.to_csv('../chemspace/Dataset/Data/CID-Synonym-filtered.tsv', sep='\t', index=False)

In [8]:
def add_SMILES(self, data_path: str = '../chemspace/Dataset/Data/CID-SMILES.gz'):
        concat_df = pd.DataFrame()
        
        # create object to iterate through CSV chunks
        data_reader = pd.read_csv(data_path, chunksize= 10 ** 6, sep= "\t", names = ['CID', 'SMILES'])
        
        # Initialize counter and 
        i = 0

        # Iterate through df chunks
        for df in data_reader:
            # Display progress
            if i % 5 == 0:
                print(i)

            # if the lowest CID of the chunk is greater than the largest CID we're interested in, then stop
            if df['CID'].iloc[0] > self.CIDs.iloc[-1]:
                return
            # Otherwise, merge the CIDs and SMILES chunk dataframes, and concat. Store as dataset
            elif self._external_CIDs_in_dataset(df['CID']):
                merged_df = self.CIDs.to_frame().merge(df, how='inner', left_on = 'CID', right_on='CID', suffixes= (None,'_y'))
                concat_df = pd.concat([concat_df, merged_df], axis = 0, ignore_index=True)
            
            # Advance counter
            i = i + 1
            self.dataset = concat_df
        return

In [9]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5524 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CID                 5524 non-null   int64 
 1   Synonyms            5524 non-null   object
 2   Number_of_Synonyms  5524 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 129.6+ KB


In [10]:
# Load df
df = pd.read_csv('../chemspace/Dataset/Data/Dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327757 entries, 0 to 327756
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   CID     327757 non-null  int64 
 1   SMILES  327757 non-null  object
dtypes: int64(1), object(1)
memory usage: 5.0+ MB


In [11]:

# Assuming df and df_grouped are your dataframes

# Initialize an empty dataframe to store the final result
result = pd.DataFrame()

# Number of rows to process at a time
chunk_size = 1000

# Iterate through the df_grouped dataframe in chunks
for i in range(0, len(df_grouped), chunk_size):
    # Get the current chunk from df_grouped
    chunk = df_grouped.iloc[i:i + chunk_size]
    
    # Merge the chunk with df based on CID
    merged_chunk = df.merge(chunk, how='inner', on='CID')
    
    # Append the merged chunk to the result dataframe
    result = pd.concat([result, merged_chunk], ignore_index=True)

# result dataframe now contains the merged data
print(result)


       CID                                         SMILES  \
0        1               CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C   
1        3                       C1=CC(C(C(=C1)C(=O)O)O)O   
2        4                                        CC(CN)O   
3        5                           C(C(=O)COP(=O)(O)O)N   
4        6      C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl   
...    ...                                            ...   
3583  6658                                  CC(=C)C(=O)OC   
3584  6659                                 COC(=O)C(=C)Cl   
3585  6660                               CC1=C(C(=O)CC1)O   
3586  6661                                  CN1CCN(C1=O)C   
3587  6662  CC1=C(ON=C1C)N(C(=O)C)S(=O)(=O)C2=CC=C(C=C2)N   

                                               Synonyms  Number_of_Synonyms  
0     (+/-)-acetylcarnitine; (3-CARBOXY-2-HYDROXYPRO...                  38  
1     100459-00-5; 2,3-dihydro-2,3-dihydroxybenzoic ...                   9  
2     (+-)-1-amino-2-propanol; (+

In [7]:
#write result to csv
result.to_csv('../chemspace/Dataset/Data/Dataset-with-Synonyms.csv', index=False)

In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250097 entries, 0 to 250096
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   CID                 250097 non-null  int64 
 1   SMILES              250097 non-null  object
 2   Synonyms            250097 non-null  object
 3   Number_of_Synonyms  250097 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.6+ MB


In [9]:
result.head()

,CID,SMILES,Synonyms,Number_of_Synonyms
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,(+/-)-acetylcarnitine| (3-CARBOXY-2-HYDROXYPRO...,38
1,3,C1=CC(C(C(=C1)C(=O)O)O)O,"100459-00-5| 2,3-dihydro-2,3-dihydroxybenzoic ...",9
2,4,CC(CN)O,(+-)-1-amino-2-propanol| (+/-)-1-Amino-2-propa...,122
3,5,C(C(=O)COP(=O)(O)O)N,(3-amino-2-oxopropyl) dihydrogen phosphate| 1-...,11
4,6,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"1 - Chloro - 2,4 - dinitrobenzene| 1,3-Dinitro...",101
